# Extract barrier island metrics along transects

Author: Emily Sturdivant, esturdivant@usgs.gov

***

Extract barrier island metrics along transects for Barrier Island Geomorphology Bayesian Network. See the project [README](https://github.com/esturdivant-usgs/BI-geomorph-extraction/blob/master/README.md) and the Methods Report (Zeigler et al., in review). 


## Pre-requisites:
- All the input layers (transects, shoreline, etc.) must be ready. This is performed with the notebook file prepper.ipynb.
- The files servars.py and configmap.py may need to be updated for the current dataset.

## Notes:
- This notebook includes interactive quality checking, which requires the user's attention. For thorough QC'ing, we recommend displaying the layers in ArcGIS, especially to confirm the integrity of values for variables such as distance to inlet (__Dist2Inlet__) and widths of the landmass (__WidthPart__, etc.). 


***

## Import modules

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import arcpy
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
import core.functions_warcpy as fwa
import core.functions as fun

### Initialize variables

This cell prompts you for the site, year, and project directory path. `setvars.py` retrieves the pre-determined values for that site in that year from `configmap.py`. The project directory will be used to set up your workspace. It's hidden for security – sorry! I recommend that you type the path somewhere and paste it in.

In [2]:
from core.setvars import *

site (options: Assawoman, CapeLookout, Monomoy, Assateague, Rockaway, RhodeIsland, Forsythe, CoastGuard, Parramore, ParkerRiver, Smith, Cobb, Cedar, FireIsland, Fisherman): RhodeIsland
year (options: 2010, 2012, 2014): 2014
Path to project directory (e.g. \\Macolume\dir\FireIsland2014): ········
setvars.py initialized variables.
SITE: RhodeIsland
MHW: 0.29
MLW: -0.42
Max dune crest height: 2.5
Projection code: 26919



Change the filename variables to match your local files. They should be in an Esri file geodatabase named site+year.gdb in your project directory, which you input above and is the value of the variable `home`. 

In [3]:
# Extended transects: NASC transects extended and sorted, ready to be the base geometry for processing
extendedTrans = os.path.join(home, 'extTrans')

# Tidied transects: Extended transects without overlapping transects
extTrans_tidy = os.path.join(home, 'tidyTrans') 

# Geomorphology points: positions of indicated geomorphic features
ShorelinePts = os.path.join(home, 'SLpts')  # shoreline
dlPts = os.path.join(home, 'DLpts')         # dune toe
dhPts = os.path.join(home, 'DCpts')         # dune crest

# Inlet lines: polyline feature classes delimiting inlet position. Must intersect the full island shoreline
inletLines = os.path.join(home, 'inletLines')

# Full island shoreline: polygon that outlines the island shoreline, MHW on oceanside and MTL on bayside
barrierBoundary = os.path.join(home, 'bndpoly_2sl_utm19')  

# Elevation grid: DEM of island elevation at either 5 m or 1 m resolution
elevGrid = os.path.join(home, 'dem_download_merged_5m')

# ---
# OPTIONAL - comment out each one that is not available
# ---
# 
# morphdata_prefix = '14CNT01'

# Study area boundary; manually digitize if the barrier island study area does not end at an inlet.
SA_bounds = os.path.join(home, 'SA_bounds')

# Armoring lines: digitize lines of shorefront armoring to be used if dune toe points are not available.
armorLines = os.path.join(home, 'armorLines')

# # Extended transects with Construction, Development, and Nourishment coding
# tr_w_anthro = os.path.join(home, 'extTrans_wAnthro')

# # Piping Plover Habitat BN raster layers
# SubType = os.path.join(home, 'SubType')   # substrate type
# VegType = os.path.join(home, 'FI12_VegType')   # vegetation type
# VegDens = os.path.join(home, 'FI12_VegDen')    # vegetation density
# GeoSet = os.path.join(home, 'FI12_GeoSet')     # geomorphic setting
# DisMOSH = os.path.join(home, 'FI12_DisMOSH')   # distance to moist substrates (MOSH)

# Derivatives of inputs: They will be generated during process if they are not found. 
shoreline = os.path.join(home, 'ShoreBetweenInlets')   # oceanside shoreline between inlets; generated from shoreline polygon, inlet lines, and SA bounds
slopeGrid = os.path.join(home, 'slope_5m')   # Slope at 5 m resolution; generated from DEM

### Split features between MHW zones - mhw29, mhw22, mhw36

In [4]:
# mhw29
extendedTrans_mhw29 = os.path.join(home, 'extTrans_mhw29')
ShorelinePts_mhw29 = os.path.join(home, 'SLpts_mhw29')
dlPts_mhw29 = os.path.join(home, 'DTpts_mhw29')
dhPts_mhw29 = os.path.join(home, 'DCpts_mhw29')
armorLines_mhw29 = os.path.join(home, 'armorLines_mhw29')
# shoreline_west = os.path.join(home, 'ShoreBetweenInlets_mhw29')

# mhw22
extendedTrans_mhw22 = os.path.join(home, 'extTrans_mhw22')
ShorelinePts_mhw22 = os.path.join(home, 'SLpts_mhw22')
dlPts_mhw22 = os.path.join(home, 'DTpts_mhw22')
dhPts_mhw22 = os.path.join(home, 'DCpts_mhw22')
armorLines_mhw22 = os.path.join(home, 'armorLines_mhw22')
# shoreline_mhw22 = os.path.join(home, 'ShoreBetweenInlets_mhw22')

# mhw36
extendedTrans_mhw36 = os.path.join(home, 'extTrans_mhw36')
ShorelinePts_mhw36 = os.path.join(home, 'SLpts_mhw36')
dlPts_mhw36 = os.path.join(home, 'DTpts_mhw36')
dhPts_mhw36 = os.path.join(home, 'DCpts_mhw36')
armorLines_mhw36 = os.path.join(home, 'armorLines_mhw36')
# shoreline_mhw36 = os.path.join(home, 'ShoreBetweenInlets_mhw36')

## Transect-averaged values
We work with the shapefile/feature class as a pandas DataFrame as much as possible to speed processing and minimize reliance on the ArcGIS GUI display.

1. Create a pandas dataframe from the transects feature class. In the process, we remove some of the unnecessary fields. The resulting dataframe is indexed by __sort_ID__ with columns corresponding to the attribute fields in the transects feature class. 
2. Add __DD_ID__.
3. Join the values from the transect file that includes the three anthropologic development fields, __Construction__,  __Development__, and __Nourishment__. 

In [5]:
# Copy feature class to dataframe.
trans_df = fwa.FCtoDF(extendedTrans, id_fld=tID_fld, extra_fields=extra_fields)
trans_df['DD_ID'] = trans_df[tID_fld] + sitevals['id_init_val']

# Get anthro fields and join to DF
if 'tr_w_anthro' in locals():
    trdf_anthro = fwa.FCtoDF(tr_w_anthro, id_fld=tID_fld, dffields=['Development', 'Nourishment','Construction'])
    trans_df = fun.join_columns(trans_df, trdf_anthro) 

# Save
trans_df.to_pickle(os.path.join(scratch_dir, 'trans_df.pkl'))

# Display
print("\nHeader of transects dataframe (rows 1-5 out of {}): ".format(len(trans_df)))
trans_df.head()

...converting feature class to array...
...converting array to dataframe...

Header of transects dataframe (rows 1-5 out of 824): 


,Azimuth,LRR,TransOrder,TransectId,sort_ID,DD_ID
sort_ID,,,,,,
1,50.12,-0.06,1.0,1.0,1,50001
2,50.12,-0.03,2.0,2.0,2,50002
3,50.12,-0.04,3.0,3.0,3,50003
4,41.89,-0.18,4.0,4.0,4,50004
5,27.67,-0.17,5.0,5.0,5,50005


In [6]:
# Sort by ID before selecting by index
trans_df = trans_df.sort_values(by=tID_fld)

# Select transects with MHW 0.29
trans_mhw29 = trans_df.loc[:117, :]
trans_mhw29.to_pickle(os.path.join(scratch_dir, 'trans_mhw29.pkl'))

# Select transects with MHW 0.22
trans_mhw22 = trans_df.loc[118:725, :]
trans_mhw22.to_pickle(os.path.join(scratch_dir, 'trans_mhw22.pkl'))

# Select transects with MHW 0.36
trans_mhw36 = trans_df.loc[726:, :]
trans_mhw36.to_pickle(os.path.join(scratch_dir, 'trans_mhw36.pkl'))

In [7]:
# Read original
trans_df = pd.read_pickle(os.path.join(scratch_dir, 'trans_df.pkl'))
trans_mhw29 = pd.read_pickle(os.path.join(scratch_dir, 'trans_mhw29.pkl'))
trans_mhw22 = pd.read_pickle(os.path.join(scratch_dir, 'trans_mhw22.pkl'))
trans_mhw36 = pd.read_pickle(os.path.join(scratch_dir, 'trans_mhw36.pkl'))
print("Length of trans_df: ", len(trans_df))
print("Length of trans_mhw29: ", len(trans_mhw29))
print("Length of trans_mhw22: ", len(trans_mhw22))
print("Length of trans_mhw36: ", len(trans_mhw36))

Length of trans_df:  824
Length of trans_mhw29:  117
Length of trans_mhw22:  608
Length of trans_mhw36:  99


### Get XY and Z/slope from SL, DH, DL points within 25 m of transects
Add to each transect row the positions of the nearest pre-created beach geomorphic features (shoreline, dune toe, and dune crest).

#### If needed, convert morphology points stored locally to feature classes for use.
After which, view the new feature classes in a GIS. Isolate the points to the region of interest. Quality check them. Then copy them for use with this code, which will require setting the filenames to match those included here or changing the values included here to match the final filenames.

In [8]:
if "morphdata_prefix" in locals():
    csvpath = os.path.join(proj_dir, 'Input_Data', '{}_morphology'.format(morphdata_prefix), 
                           '{}_morphology.csv'.format(morphdata_prefix))
    dt_fc, dc_fc, sl_fc = fwa.MorphologyCSV_to_FCsByFeature(csvpath, state, proj_code, 
                                                            csv_fill = 999, fc_fill = -99999, csv_epsg=4326)
    print("OUTPUT: morphology point feature classes in the scratch gdb. We recommend QC before proceeding.")

#### Shoreline

The MHW shoreline easting and northing (__SL_x__, __SL_y__) are the coordinates of the intersection of the oceanside shoreline with the transect. Each transect is assigned the foreshore slope (__Bslope__) from the nearest shoreline point within 25 m. These values are populated for each transect as follows: 
1. get __SL_x__ and __SL_y__ at the point where the transect crosses the oceanside shoreline; 
2. find the closest shoreline point to the intersection point (must be within 25 m) and copy the slope value from the point to the transect in the field __Bslope__.

In [9]:
#%% Shoreline
# mhw29: Get the XY position where transect crosses the oceanside shoreline
sl2trans_mhw29 = fwa.add_shorelinePts2Trans(extendedTrans_mhw29, ShorelinePts_mhw29, shoreline, 
                                            tID_fld, proximity=pt2trans_disttolerance)
sl2trans_mhw29.to_pickle(os.path.join(scratch_dir, 'sl2trans_mhw29.pkl'))


# mhw29: Get the XY position where transect crosses the oceanside shoreline
sl2trans_mhw22 = fwa.add_shorelinePts2Trans(extendedTrans_mhw22, ShorelinePts_mhw22, shoreline, 
                                            tID_fld, proximity=pt2trans_disttolerance)
sl2trans_mhw22.to_pickle(os.path.join(scratch_dir, 'sl2trans_mhw22.pkl'))


# mhw36: Get the XY position where transect crosses the oceanside shoreline
sl2trans_mhw36 = fwa.add_shorelinePts2Trans(extendedTrans_mhw36, ShorelinePts_mhw36, shoreline,
                                            tID_fld, proximity=pt2trans_disttolerance)
sl2trans_mhw36.to_pickle(os.path.join(scratch_dir, 'sl2trans_mhw36.pkl'))


Matching shoreline points to transects...
Using field 'slope' as slope.
The projection of SLpts_mhw29 was changed. The new file is SLpts_mhw29_utm.
...duration at transect 100: 0:0:32.5 seconds
Duration: 0:0:36.7 seconds

Matching shoreline points to transects...
Using field 'slope' as slope.
The projection of SLpts_mhw22 was changed. The new file is SLpts_mhw22_utm.
...duration at transect 200: 0:0:42.8 seconds
...duration at transect 300: 0:1:28.1 seconds
...duration at transect 400: 0:2:26.0 seconds
...duration at transect 500: 0:3:12.0 seconds
...duration at transect 600: 0:4:7.8 seconds
...duration at transect 700: 0:4:58.2 seconds
Duration: 0:5:10.7 seconds

Matching shoreline points to transects...
Using field 'slope' as slope.
The projection of SLpts_mhw36 was changed. The new file is SLpts_mhw36_utm.
...duration at transect 800: 0:0:38.5 seconds
Duration: 0:0:48.1 seconds


#### Dune positions along transects

__DL_x__, __DL_y__, and __DL_z__ are the easting, northing, and elevation, respectively, of the nearest dune toe point within 25 meters of the transect. __DH_x__, __DH_y__, and __DH_z__ are the easting, northing, and elevation, respectively, of the nearest dune crest point within 25 meters. 

__DL_snapX__, __DL_snapY__, __DH_snapX__, and __DH_snapY__ are the eastings and northings of the points "snapped" to the transect. "Snapping" finds the position along the transect nearest to the point, i.e. orthogonal to the transect. These values are used to find the beach width. The elevation values are not snapped; we use the elevation values straight from the original points. 

These values are populated as follows: 

1. Find the nearest dune crest/toe point to the transect and proceed if the distance is less than 25 m. If there are no points within 25 m of the transect, populate the row with Null values.
2. Get the X, Y, and Z values of the point. 
3. Find the position along the transect of an orthogonal line drawn to the dune point (__DL_snapX__, __DL_snapY__, __DH_snapX__, and __DH_snapY__). This is considered the 'snapped' XY position and is calculated using the arcpy geometry method. 

In [10]:
# mhw29: Create dataframe for both dune crest and dune toe positions
dune2trans_mhw29 = fwa.find_ClosestPt2Trans_snap(extendedTrans_mhw29, dhPts_mhw29, dlPts_mhw29, trans_mhw29, 
                                          tID_fld, proximity=pt2trans_disttolerance)
dune2trans_mhw29.to_pickle(os.path.join(scratch_dir, 'dune2trans_mhw29.pkl'))

# mhw22: Create dataframe for both dune crest and dune toe positions
dune2trans_mhw22 = fwa.find_ClosestPt2Trans_snap(extendedTrans_mhw22, dhPts_mhw22, dlPts_mhw22, trans_mhw22, 
                                          tID_fld, proximity=pt2trans_disttolerance)
dune2trans_mhw22.to_pickle(os.path.join(scratch_dir, 'dune2trans_mhw22.pkl'))

# mhw36: Create dataframe for both dune crest and dune toe positions
dune2trans_mhw36 = fwa.find_ClosestPt2Trans_snap(extendedTrans_mhw36, dhPts_mhw36, dlPts_mhw36, trans_mhw36, 
                                          tID_fld, proximity=pt2trans_disttolerance)
dune2trans_mhw36.to_pickle(os.path.join(scratch_dir, 'dune2trans_mhw36.pkl'))


Matching dune points with transects:
Using field 'dhigh_z' as DH Z field...
The projection of DCpts_mhw29 was changed. The new file is DCpts_mhw29_utm.
Using field 'dlow_z' as DL Z field...
The projection of DTpts_mhw29 was changed. The new file is DTpts_mhw29_utm.
Looping through transects and dune points to find nearest point within 25 m...
...duration at transect 100: 0:1:4.6 seconds
Duration: 0:1:9.2 seconds

Matching dune points with transects:
Using field 'dhigh_z' as DH Z field...
The projection of DCpts_mhw22 was changed. The new file is DCpts_mhw22_utm.
Using field 'dlow_z' as DL Z field...
The projection of DTpts_mhw22 was changed. The new file is DTpts_mhw22_utm.
Looping through transects and dune points to find nearest point within 25 m...
...duration at transect 200: 0:1:28.6 seconds
...duration at transect 300: 0:2:46.2 seconds
...duration at transect 400: 0:4:2.7 seconds
...duration at transect 500: 0:5:7.3 seconds
...duration at transect 600: 0:6:9.6 seconds
...duratio

#### Armoring
__Arm_x__, __Arm_y__, and __Arm_z__ are the easting, northing, and elevation, respectively, where an artificial structure crosses the transect in the vicinity of the beach. These features are meant to supplement the dune toe data set by providing an upper limit to the beach in areas where dune toe extraction was confounded by the presence of an artificial structure. Values are populated for each transect as follows: 

1. Get the positions of intersection between the digitized armoring lines and the transects (Intersect tool from the Overlay toolset); 
2. Extract the elevation value at each intersection point from the DEM (Extract Multi Values to Points tool from Spatial Analyst); 

In [12]:
# Create elevation raster at 5-m resolution if not already
elevGrid = os.path.join(home, 'dem_download_merged_5m')
elevGrid = fwa.ProcessDEM(elevGrid, utmSR)

# mhw29
arm2trans_mhw29 = fwa.ArmorLineToTrans_PD(extendedTrans_mhw29, armorLines_mhw29, sl2trans_mhw29, tID_fld, proj_code, elevGrid)
arm2trans_mhw29.to_pickle(os.path.join(scratch_dir, 'arm2trans_mhw29.pkl'))
print('Completed processing for MHW 0.29')

arm2trans = os.path.join(arcpy.env.scratchGDB, "arm2trans")
arm2trans_new = os.path.join(arcpy.env.scratchGDB, "arm2trans_mhw29")
arcpy.CopyFeatures_management(arm2trans, arm2trans_new)
arcpy.Delete_management(arm2trans)

OUTPUT: dem_download_merged_5m_5m at 5x5 resolution.
Getting elevation of beach armoring by extracting elevation values to arm2trans points.
Converting feature class to array with X and Y...
...converting array to dataframe...
Looks like these transects [60] are intersected by armoring lines multiple times. We will select the more seaward of the points.
Completed processing for MHW 0.29


<Result 'true'>

In [13]:
# mhw29
arm2trans_mhw22 = fwa.ArmorLineToTrans_PD(extendedTrans_mhw22, armorLines_mhw22, sl2trans_mhw22, tID_fld, proj_code, elevGrid)
arm2trans_mhw22.to_pickle(os.path.join(scratch_dir, 'arm2trans_mhw22.pkl'))
print('Completed processing for MHW 0.22')

arm2trans_new = os.path.join(arcpy.env.scratchGDB, "arm2trans_mhw22")
arcpy.CopyFeatures_management(arm2trans, arm2trans_new)
arcpy.Delete_management(arm2trans)

Converting feature class to array with X and Y...
...converting array to dataframe...
Looks like these transects [60] are intersected by armoring lines multiple times. We will select the more seaward of the points.


KeyError: 'the label [60] is not in the [index]'

In [ ]:
# mhw36
arm2trans_mhw36 = fwa.ArmorLineToTrans_PD(extendedTrans_mhw36, armorLines_mhw36, sl2trans_mhw36, tID_fld, proj_code, elevGrid)
arm2trans_mhw36.to_pickle(os.path.join(scratch_dir, 'arm2trans_mhw36.pkl'))
print('Completed processing for MHW 0.36')

arm2trans_new = os.path.join(arcpy.env.scratchGDB, "arm2trans_mhw36")
arcpy.CopyFeatures_management(arm2trans, arm2trans_new)
arcpy.Delete_management(arm2trans)

### Add all the positions to the trans_df
Join the new dataframes to the transect dataframe. Before it performs the join, `join_columns_id_check()` checks the index and the ID field for potential errors such as whether they are the equal and whether there are duplicated IDs or null values in either.

In [ ]:
# Load saved dataframes
trans_df = pd.read_pickle(os.path.join(scratch_dir, 'trans_df.pkl'))
trans_mhw29 = pd.read_pickle(os.path.join(scratch_dir, 'trans_mhw29.pkl'))
trans_mhw22 = pd.read_pickle(os.path.join(scratch_dir, 'trans_mhw22.pkl'))
trans_mhw36 = pd.read_pickle(os.path.join(scratch_dir, 'trans_mhw36.pkl'))

sl2trans_mhw29 = pd.read_pickle(os.path.join(scratch_dir, 'sl2trans_mhw29.pkl'))
sl2trans_mhw22 = pd.read_pickle(os.path.join(scratch_dir, 'sl2trans_mhw22.pkl'))
sl2trans_mhw36 = pd.read_pickle(os.path.join(scratch_dir, 'sl2trans_mhw36.pkl'))

dune2trans_mhw29 = pd.read_pickle(os.path.join(scratch_dir, 'dune2trans_mhw29.pkl'))
dune2trans_mhw22 = pd.read_pickle(os.path.join(scratch_dir, 'dune2trans_mhw22.pkl'))
dune2trans_mhw36 = pd.read_pickle(os.path.join(scratch_dir, 'dune2trans_mhw36.pkl'))

arm2trans_mhw29 = pd.read_pickle(os.path.join(scratch_dir, 'arm2trans_mhw29.pkl'))
arm2trans_mhw22 = pd.read_pickle(os.path.join(scratch_dir, 'arm2trans_mhw22.pkl'))
arm2trans_mhw36 = pd.read_pickle(os.path.join(scratch_dir, 'arm2trans_mhw36.pkl'))

In [ ]:
# West: Join positions of shoreline, dune crest, dune toe, armoring
trans_mhw29 = fun.join_columns_id_check(trans_mhw29, sl2trans_mhw29, tID_fld)
trans_mhw29 = fun.join_columns_id_check(trans_mhw29, dune2trans_mhw29, tID_fld)
trans_mhw29 = fun.join_columns_id_check(trans_mhw29, arm2trans_mhw29, tID_fld)
# Save
trans_mhw29.to_pickle(os.path.join(scratch_dir, 'trans_mhw29_beachmetrics.pkl'))

# East: Join positions of shoreline, dune crest, dune toe, armoring
trans_mhw22 = fun.join_columns_id_check(trans_mhw22, sl2trans_mhw22, tID_fld)
trans_mhw22 = fun.join_columns_id_check(trans_mhw22, dune2trans_mhw22, tID_fld)
trans_mhw22 = fun.join_columns_id_check(trans_mhw22, arm2trans_mhw22, tID_fld)
# Save
trans_mhw22.to_pickle(os.path.join(scratch_dir, 'trans_mhw22_beachmetrics.pkl'))

# East: Join positions of shoreline, dune crest, dune toe, armoring
trans_mhw36 = fun.join_columns_id_check(trans_mhw36, sl2trans_mhw36, tID_fld)
trans_mhw36 = fun.join_columns_id_check(trans_mhw36, dune2trans_mhw36, tID_fld)
trans_mhw36 = fun.join_columns_id_check(trans_mhw36, arm2trans_mhw36, tID_fld)
# Save
trans_mhw36.to_pickle(os.path.join(scratch_dir, 'trans_mhw36_beachmetrics.pkl'))


In [ ]:
# Concatenate the transect dataframes
trans_all_df = pd.concat([trans_mhw29, trans_mhw22, trans_mhw36])
trans_all_df.to_pickle(os.path.join(scratch_dir, 'trans_df_beachmetrics_concat.pkl'))
trans_all_df.head()

# Concatenate the morphology dataframes
sl2trans_df = pd.concat([sl2trans_mhw29, sl2trans_mhw22, sl2trans_mhw36])
sl2trans_df.to_pickle(os.path.join(scratch_dir, 'sl2trans_concat.pkl'))

dune2trans_df = pd.concat([dune2trans_mhw29, dune2trans_mhw22, dune2trans_mhw36])
dune2trans_df.to_pickle(os.path.join(scratch_dir, 'dune2trans_concat.pkl'))

arm2trans_df = pd.concat([arm2trans_mhw29, arm2trans_mhw22, arm2trans_mhw36])
arm2trans_df.to_pickle(os.path.join(scratch_dir, 'arm2trans_df_concat.pkl'))

### Check for errors
*Optional*

Display summary stats / histograms and create feature classes. The feature classes display the locations that will be used to calculate beach width. Review the output feature classes in a GIS to validate. 

In [ ]:
plots = trans_df.hist(['DH_z', 'DL_z', 'Arm_z'])

# Subplot Labels
plots[0][0].set_xlabel("Elevation (m in NAVD88)")
plots[0][0].set_ylabel("Frequency")
plots[0][1].set_xlabel("Elevation (m in NAVD88)")
plots[0][1].set_ylabel("Frequency")
try:
    plots[0][2].set_xlabel("Elevation (m in NAVD88)")
    plots[0][2].set_ylabel("Frequency")
except:
    pass

plt.show()
plt.close()

In [ ]:
# Convert dataframe to feature class - shoreline points with slope
fwa.DFtoFC(sl2trans_df, os.path.join(arcpy.env.workspace, 'pts2trans_SL'), 
           spatial_ref=utmSR, id_fld=tID_fld, xy=["SL_x", "SL_y"], keep_fields=['Bslope'])
print('OUTPUT: pts2trans_SL in designated scratch geodatabase.')

# Dune crests
try:
    fwa.DFtoFC(dune2trans_df, os.path.join(arcpy.env.workspace, 'ptSnap2trans_DH'), 
               spatial_ref=utmSR, id_fld=tID_fld, xy=["DH_snapX", "DH_snapY"], keep_fields=['DH_z'])
    print('OUTPUT: ptSnap2trans_DH in designated scratch geodatabase.')
except Exception as err:
    print(err)
    pass

# Dune toes
try:
    fwa.DFtoFC(dune2trans_df, os.path.join(arcpy.env.workspace, 'ptSnap2trans_DL'), 
               spatial_ref=utmSR, id_fld=tID_fld, xy=["DL_snapX", "DL_snapY"], keep_fields=['DL_z'])
    print('OUTPUT: ptSnap2trans_DL in designated scratch geodatabase.')
except Exception as err:
    print(err)
    pass

### Calculate upper beach width and height
Upper beach width (__uBW__) and upper beach height (__uBH__) are calculated based on the difference in position between two points: the position of MHW along the transect (__SL_x__, __SL_y__) and the dune toe position or equivalent (usually __DL_snapX__, __DL_snapY__).  In some cases, the dune toe is not appropriate to designate the "top of beach" so beach width and height are calculated from either the position of the dune toe, the dune crest, or the base of an armoring structure. The dune crest was only considered a possibility if the dune crest elevation (__DH_zMHW__) was less than or equal to `maxDH`. 

They are calculated as follows: 
2. Calculate distances from MHW to the position along the transect of the dune toe (__DistDL__), dune crest (__DistDH__), and armoring (__DistArm__). 
2. Adjust the elevations to MHW, populating fields __DH_zmhw__, __DL_zmhw__, and __Arm_zmhw__. 
3. Conditionally select the appropriate feature to represent "top of beach." Dune toe is prioritized. If it is not available and __DH_zmhw__ is less than or equal to maxDH, use dune crest. If neither of the dune positions satisfy the conditions and an armoring feature intersects with the transect, use the armoring position. If none of the three are possible, __uBW__ and __uBH__ will be null. 
4. Copy the distance to shoreline and height above MHW (__Dist--__, __---zmhw__) to __uBW__ and __uBH__, respectively. 

Notes:
- In some morphology datasets, missing elevation values at a point indicate that the point should not be used to measure beach width. In those cases, use the `skip_missing_z` argument to select whether or not to skip these points. 

In [ ]:
# Load saved dataframe
trans_df = pd.read_pickle(os.path.join(scratch_dir, 'trans_df_beachmetrics_concat.pkl'))

In [ ]:
# Calculate distances from shore to dunes, etc.
# beach width calc loop will only update transect rows in trans_df with tIDs in extendedTrans

MHW = 0.29
trans_df  = fwa.calc_BeachWidth_fill(extendedTrans_mhw29, trans_df, maxDH, tID_fld, MHW, fill, skip_missing_z=True)

MHW = 0.22
trans_df  = fwa.calc_BeachWidth_fill(extendedTrans_mhw22, trans_df, maxDH, tID_fld, MHW, fill, skip_missing_z=True)

MHW = 0.36
trans_df  = fwa.calc_BeachWidth_fill(extendedTrans_mhw36, trans_df, maxDH, tID_fld, MHW, fill, skip_missing_z=True)

### Dist2Inlet


Distance to nearest tidal inlet (__Dist2Inlet__) is computed as alongshore distance of each sampling transect from the nearest tidal inlet. This distance includes changes in the path of the shoreline instead of simply a Euclidean distance and reflects sediment transport pathways. It is measured using the oceanside shoreline between inlets (ShoreBetweenInlets). 

Note that the ShoreBetweenInlets feature class must be both 'dissolved' and 'singlepart' so that each feature represents one-and-only-one shoreline that runs the entire distance between two inlets or equivalent. If the shoreline is bounded on both sides by an inlet, measure the distance to both and assign the minimum distance of the two. If the shoreline meets only one inlet (meaning the study area ends before the island ends), use the distance to the only inlet. 

The process uses the cut, disjoint, and length geometry methods and properties in ArcPy data access module. The function measure_Dist2Inlet() prints a warning when the difference in Dist2Inlet between two consecutive transects is greater than 300. 

In [ ]:
# Calc Dist2Inlet in new dataframe 
dist_df = fwa.measure_Dist2Inlet(shoreline, extendedTrans, inletLines, tID_fld)

# Join to transects
trans_df = fun.join_columns_id_check(trans_df, dist_df, tID_fld, fill=fill)

# Save and view last 10 rows
dist_df.to_pickle(os.path.join(scratch_dir, 'dist2inlet_df.pkl'))
dist_df.tail(10)

### Clip transects, get barrier widths
Calculates __WidthLand__, __WidthFull__, and __WidthPart__, which measure different flavors of the cross-shore width of the barrier island. __WidthLand__ is the above-water distance between the back-barrier and seaward MHW shorelines. __WidthLand__ only includes regions of the barrier within the shoreline polygon (bndpoly_2sl) and does not extend into any of the sinuous or intervening back-barrier waterways and islands. __WidthFull__ is the total distance between the back-barrier and seaward MHW shorelines (including space occupied by waterways). __WidthPart__ is the width of only the most seaward portion of land within the shoreline. 

These are calculated as follows: 

1. Clip the transect to the full island shoreline (Clip in the Analysis toolbox); 
2. For __WidthLand__, get the length of the multipart line segment from &quot;SHAPE@LENGTH&quot; feature class attribute. When the feature is multipart, this will include only the remaining portions of the transect; 
3. For __WidthPart__, convert the clipped transect from multipart to singlepart and get the length of the first line segment, which should be the most seaward; 
4. For __WidthFull__, calculate the distance between the first vertex and the last vertex of the clipped transect (Feature Class to NumPy Array with explode to points, pandas groupby, numpy hypot).

In [ ]:
# Clip transects, get barrier widths
widths_df = fwa.calc_IslandWidths(extendedTrans, barrierBoundary, tID_fld=tID_fld)

# # Save
widths_df.to_pickle(os.path.join(scratch_dir, 'widths_df.pkl'))

# Join
trans_df = fun.join_columns_id_check(trans_df, widths_df, tID_fld, fill=fill)

# Save
trans_df.to_pickle(os.path.join(scratch_dir, trans_name+'_null_prePts.pkl'))
trans_df.sample(5)

## 5-m Points
The point dataset samples the land every 5 m along each shore-normal transect. 

### Split transects into points at 5-m intervals. 

The point dataset is created from the tidied transects (tidyTrans, created during pre-processing) as follows: 

1. Clip the tidied transects (tidyTrans) to the shoreline polygon (bndpoly_2sl) , retaining only those portions of the transects that represent land.
2. Produce a dataframe of point positions along each transect every 5 m starting from the ocean-side shoreline. This uses the positionAlongLine geometry method accessed with a Search Cursor and saves the outputs in a new dataframe. 
3. Create a point feature class from the dataframe. 

In [ ]:
pts_df, pts_presort = fwa.TransectsToPointsDF(extTrans_tidy, barrierBoundary, fc_out=pts_presort)
print("OUTPUT: '{}' in scratch geodatabase.".format(os.path.basename(pts_presort)))

# Save
pts_df.to_pickle(os.path.join(scratch_dir, 'pts_presort.pkl'))

### Add Elevation and Slope to points

__ptZ__ (later __ptZmhw__) and __ptSlp__ are the elevation and slope at the 5-m cell corresponding to the point. 
1. Create the slope and DEM rasters if they don't already exist. We use the 5-m DEM to generate a slope surface (Slope tool in 3D Analyst). 
2. Use Extract Multi Values to Points tool in Spatial Analyst. 
3. Convert the feature class back to a dataframe.

In [ ]:
# Create slope raster from DEM
if not arcpy.Exists(slopeGrid):
    arcpy.Slope_3d(elevGrid, slopeGrid, 'PERCENT_RISE')
    print("OUTPUT: slope file in designated home geodatabase.")
    
# Add elevation and slope values at points.
arcpy.sa.ExtractMultiValuesToPoints(pts_presort, [[elevGrid, 'ptZ'], [slopeGrid, 'ptSlp']])
print("OUTPUT: added slope and elevation to '{}' in designated scratch geodatabase.".format(os.path.basename(pts_presort)))

# Save
pts_df.to_pickle(os.path.join(scratch_dir, 'pts_extractedZ_presort.pkl'))

In [ ]:
if 'SubType' in locals():
    # Add substrate type, geomorphic setting, veg type, veg density values at points.
    arcpy.sa.ExtractMultiValuesToPoints(pts_presort, [[SubType, 'SubType'], [VegType, 'VegType'], 
                                                           [VegDens, 'VegDens'], [GeoSet, 'GeoSet'],
                                                            [DisMOSH, 'DisMOSH']])

    # Convert to dataframe
    pts_df = fwa.FCtoDF(pts_presort, xy=True, dffields=[tID_fld,'ptZ', 'ptSlp', 'SubType', 
                                                             'VegType', 'VegDens', 'GeoSet','DisMOSH'])
    # Recode fill values
    pts_df.replace({'GeoSet': {9999:np.nan}, 'SubType': {9999:np.nan}, 'VegType': {9999:np.nan},
                    'VegDens': {9999:np.nan}, 'DisMOSH': {9999:np.nan}}, inplace=True)
else:
    print("Plover BN layers not specified (we only check for SubType), so we'll proceed without them. ")
    # Convert to dataframe
    pts_df = fwa.FCtoDF(pts_presort, xy=True, dffields=[tID_fld,'ptZ', 'ptSlp'])

# Save and view sample
pts_df.to_pickle(os.path.join(scratch_dir, 'pts_extractedvalues_presort.pkl'))
pts_df.sample(5)

In [ ]:
# Print histogram of elevation extracted to points
pts_df.hist('ptZ')

# Subplot Labels
plots[0][0].set_xlabel("Elevation (m in NAVD88)")
plots[0][0].set_ylabel("Frequency")

# Display
plt.show()
plt.close()

### Calculate distances and sort points

__SplitSort__ is a unique numeric identifier of the 5-m points at the study site, sorted by order along shoreline and by distance from oceanside. __SplitSort__ values are populated by sorting the points by __sort_ID__ and __Dist_Seg__ (see below). 

__Dist_Seg__ is the Euclidean distance between the point and the seaward shoreline (__SL_x__, __SL_y__). __Dist_MHWbay__ is the distance between the point and the bayside shoreline and is calculated by subtracting the __Dist_Seg__ value from the __WidthPart__ value of the transect. 

__DistSegDH__, __DistSegDL__, and __DistSegArm__ measure the distance of each 5-m point from the dune crest and dune toe position along a particular transect. They are calculated as the Euclidean distance between the 5-m point and the given feature. 

In [ ]:
# Load saved dataframes
pts_df = pd.read_pickle(os.path.join(scratch_dir, 'pts_extractedvalues_presort.pkl'))
trans_df = pd.read_pickle(os.path.join(scratch_dir, trans_name+'_null_prePts.pkl'))

In [ ]:
# Calculate DistSeg, Dist_MHWbay, DistSegDH, DistSegDL, DistSegArm, and sort points (SplitSort)
pts_df = fun.join_columns(pts_df, trans_df, tID_fld)
pts_df = fun.prep_points(pts_df, tID_fld, pID_fld, sitevals['MHW'], fill)

# Aggregate ptZmhw to max and mean and join to transects
pts_df, zmhw = fun.aggregate_z(pts_df, sitevals['MHW'], tID_fld, 'ptZ', fill)
trans_df = fun.join_columns(trans_df, zmhw) 

# Join transect values to pts
pts_df = fun.join_columns(pts_df, trans_df, tID_fld)

# pID_fld needs to be among the columns
if not pID_fld in pts_df.columns:
    pts_df.reset_index(drop=False, inplace=True)

# Drop extra fields and sort columns
trans_df.drop(extra_fields, axis=1, inplace=True, errors='ignore')
pts_df = pts_df.reindex_axis(sorted_pt_flds, axis=1)

# Save dataframes 
trans_df.to_pickle(os.path.join(scratch_dir, trans_name+'_null.pkl'))
pts_df.to_pickle(os.path.join(scratch_dir, pts_name+'_null.pkl'))

# View random rows from the points DF
pts_df.sample(5)

### Recode the values for CSV output and model running

In [ ]:
# Recode
pts_df4csv = pts_df.replace({'SubType': {7777:'{1111, 2222}', 1000:'{1111, 3333}'}, 
                              'VegType': {77:'{11, 22}', 88:'{22, 33}', 99:'{33, 44}'},
                              'VegDens': {666: '{111, 222}', 777: '{222, 333}', 
                                          888: '{333, 444}', 999: '{222, 333, 444}'}})

# Fill NAs
pts_df4csv.fillna(fill, inplace=True) 

# Save and view sample
pts_df4csv.to_pickle(os.path.join(scratch_dir, pts_name+'_csv.pkl'))
pts_df4csv.sample(5)

## Quality checking
Look at extracted profiles from around the island. Enter the transect ID within the available range when prompted. Evaluate the plots for consistency among variables. Repeat various times until you can be satisfied that the variables are consistent with each other and appear to represent reality. View areas with inconsistencies in a GIS.

In [ ]:
desccols = ['DL_zmhw', 'DH_zmhw', 'Arm_zmhw', 'uBW', 'uBH', 'Dist2Inlet', 
            'WidthPart', 'WidthLand', 'WidthFull', 'mean_Zmhw', 'max_Zmhw']

# Histograms
trans_df.hist(desccols, sharey=True, figsize=[15, 10], bins=20)
plt.show()
plt.close('all')

In [ ]:
flds_dist = ['SplitSort', 'Dist_Seg', 'Dist_MHWbay', 'DistSegDH', 'DistSegDL', 'DistSegArm']
flds_z = ['ptZmhw', 'ptZ', 'ptSlp']
pts_df.loc[:,flds_dist+flds_z].describe()
pts_df.hist(flds_dist, sharey=True, figsize=[15, 8], layout=(2,3))
pts_df.hist(flds_z, sharey=True, figsize=[15, 4], layout=(1,3))

plt.show()
plt.close('all')

In [ ]:
# Prompt for transect identifier (sort_ID) and get all points from that transect.
trans_in = int(input('Transect ID ("sort_ID" {:d}-{:d}): '.format(int(pts_df[tID_fld].head(1)), int(pts_df[tID_fld].tail(1)))))
pts_set = pts_df[pts_df[tID_fld] == trans_in]

# Plot
fig = plt.figure(figsize=(13,10))

# Plot the width of the island.
ax1 = fig.add_subplot(211)
try:
    fun.plot_island_profile(ax1, pts_set, sitevals['MHW'], sitevals['MTL'])
except TypeError as err:
    print('TypeError: {}'.format(err))
    pass

# Zoom in on the upper beach.
ax2 = fig.add_subplot(212)
try:
    fun.plot_beach_profile(ax2, pts_set, sitevals['MHW'], sitevals['MTL'], maxDH)
except TypeError as err:
    print('TypeError: {}'.format(err))
    pass 

# Display
plt.show()
plt.close('all')

## Outputs

### Transect-averaged
Output the transect-averaged metrics in the following formats:
- transects, unpopulated except for ID values, as gdb feature class
- transects, unpopulated except for ID values, as shapefile
- populated transects with fill values as gdb feature class
- populated transects with null values as gdb feature class
- populated transects with fill values as shapefile
- raster of beach width (__uBW__) by transect

In [ ]:
# Load the dataframe
trans_df = pd.read_pickle(os.path.join(scratch_dir, trans_name+'_null.pkl'))

#### Vector format

In [ ]:
# Create transect file with only ID values and geometry to publish.
trans_flds = ['TRANSECTID', 'TRANSORDER', 'DD_ID']
trans_4pub = fwa.JoinDFtoFC(trans_df.loc[:,trans_flds], extendedTrans, tID_fld, out_fc=sitevals['code']+'_trans')
out_shp = arcpy.FeatureClassToFeatureClass_conversion(trans_4pub, scratch_dir, sitevals['code']+'_trans.shp')
print("OUTPUT: {} in specified scratch_dir.".format(os.path.basename(str(out_shp))))

In [ ]:
# Create transect FC with fill values - Join values from trans_df to the transect FC as a new file.
trans_fc = fwa.JoinDFtoFC(trans_df, extendedTrans, tID_fld, out_fc=trans_name+'_fill')

# Create transect FC with null values
fwa.CopyFCandReplaceValues(trans_fc, fill, None, out_fc=trans_name+'_null', out_dir=home)

# Save final transect SHP with fill values
out_shp = arcpy.FeatureClassToFeatureClass_conversion(trans_fc, scratch_dir, trans_name+'_shp.shp')
print("OUTPUT: {} in specified scratch_dir.".format(os.path.basename(str(out_shp))))

#### Raster - beach width
It may be necessary to close any Arc sessions you have open.

In [ ]:
# Create a template raster corresponding to the transects. 
if not arcpy.Exists(rst_transID):
    print("{} was not found so we will create the base raster.".format(os.path.basename(rst_transID)))
    outEucAll = arcpy.sa.EucAllocation(extTrans_tidy, maximum_distance=50, cell_size=cell_size, source_field=tID_fld)
    outEucAll.save(os.path.basename(rst_transID))

# Create raster of uBW values by joining trans_df to the template raster.
out_rst = fwa.JoinDFtoRaster(trans_df, os.path.basename(rst_transID), bw_rst, fill, tID_fld, 'uBW')

### 5-m points

Output the point metrics in the following formats:
- tabular, in CSV
- populated points with fill values as gdb feature class
- populated points with null values as gdb feature class
- populated points with fill values as shapefile

In [ ]:
# Load the saved dataframes
pts_df4csv = pd.read_pickle(os.path.join(scratch_dir, pts_name+'_csv.pkl'))
pts_df = pd.read_pickle(os.path.join(scratch_dir, pts_name+'_null.pkl'))

#### Tabular format

In [ ]:
# Save CSV in scratch_dir
csv_fname = os.path.join(scratch_dir, pts_name +'.csv')
pts_df4csv.to_csv(csv_fname, na_rep=fill, index=False)
print("OUTPUT: {} in specified scratch_dir.".format(os.path.basename(csv_fname)))

#### Vector format

In [ ]:
# Convert pts_df to FC - automatically converts NaNs to fills (default fill is -99999)
pts_fc = fwa.DFtoFC_large(pts_df, out_fc=os.path.join(arcpy.env.workspace, pts_name+'_fill'), 
                          spatial_ref=utmSR, df_id=pID_fld, xy=["seg_x", "seg_y"])

# Save final FCs with null values
fwa.CopyFCandReplaceValues(pts_fc, fill, None, out_fc=pts_name+'_null', out_dir=home)

# Save final points as SHP with fill values
out_pts_shp = arcpy.FeatureClassToFeatureClass_conversion(pts_fc, scratch_dir, pts_name+'_shp.shp')
print("OUTPUT: {} in specified scratch_dir.".format(os.path.basename(str(out_pts_shp))))

### Report field values

In [ ]:
print('{}{} -- Number of points in dataset: {}'.format(site, year, pts_df4csv.shape))
for fld in pts_df4csv.columns:
    if fld in ['GeoSet', 'SubType', 'VegDens', 'VegType', 'Construction', 'Development', 'Nourishment', 'ub_feat']:
        print('{} values: {}'.format(fld, '  |  '.join(str(x) for x in pts_df4csv.loc[:,fld].unique())))
    else:
        if not any(pd.isnull(pts_df4csv.loc[:,fld])):
            mn = min(pts_df4csv.loc[:,fld])
        else:
            mn = -99999
        mx = np.max(pts_df4csv.loc[:,fld])
        print('{} min|max: {}  |  {}'.format(fld, mn, mx))